In [3]:
import serial
import time

# Arduinoとのシリアル通信設定
arduino_port = 'COM4'  # Arduinoのポートに合わせて変更
arduino_baudrate = 115200
ser = serial.Serial(arduino_port, arduino_baudrate, timeout=1)

#シリアル通信同期
try:
    # Arduinoのレスポンス待ち
    while True:
        # データを送信
        print('serial_connecting')
        ser.write(('serial_connecting' + '\n').encode())
        response = ser.readline().decode().strip()
        if response == "serial_connected":
            print("Arduino Response:", response)
            break

except Exception as e:
    print("Error:", str(e))


while True:
    print('motor_offset')
    ser.write(('motor_offset' + '\n').encode())
    response = ser.readline().decode().strip()
    if response == "processing_now":
        print("Arduino Response:", response)
        break

while True:
    response = ser.readline().decode().strip()
    if response == "processing_completed":
        print("Arduino Response:", response)
        break

time.sleep(1)

#-end--#
while True:
    print('process_end')
    ser.write(('process_end' + '\n').encode())
    response = ser.readline().decode().strip()
    if response == "end_return":
        print("Arduino Response:", response)
        break

print('------process end------')
ser.close()


serial_connecting
serial_connecting
serial_connecting
serial_connecting
serial_connecting
serial_connecting
serial_connecting
serial_connecting
serial_connecting
Arduino Response: serial_connected
motor_offset
motor_offset
Arduino Response: processing_now
Arduino Response: processing_completed
process_end
process_end
process_end
process_end
Arduino Response: end_return
------process end------


In [1]:
import cv2
from cv2 import aruco
import numpy as np
import serial
import time

def get_mark_coordinate(cap, dict_aruco, parameters, num_id):
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    corners, ids, rejectedImgPoints = aruco.detectMarkers(gray, dict_aruco, parameters=parameters)

    if ids is not None and num_id in ids:
        index = np.where(ids == num_id)[0][0]
        cornerUL = corners[index][0][0]
        cornerUR = corners[index][0][1]
        cornerBR = corners[index][0][2]
        cornerBL = corners[index][0][3]

        center = [(cornerUL[0] + cornerBR[0]) / 2, (cornerUL[1] + cornerBR[1]) / 2]

        # Draw coordinates on the frame
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(frame, f'ID: {num_id}', (int(center[0]), int(center[1])), font, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
        cv2.putText(frame, f'Coord: ({center[0]:.2f}, {center[1]:.2f})', (10, 30), font, 0.5, (0, 0, 255), 1, cv2.LINE_AA)

        cv2.imshow('Camera Window', frame)

        return center

    # If ID is not found, display the original frame
    cv2.imshow('Camera Window', frame)

    return None

def send_command_and_wait(ser, cap, command, expected_response):
    cap.grab()
    while True:
        if command != "none":
            ser.write((command + '\n').encode())

        response = ser.readline().decode().strip()
        if response == expected_response:
            print("Arduino Response:", response)
            break

cameraID = 0
cap = cv2.VideoCapture(cameraID)
dict_aruco = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)
parameters = aruco.DetectorParameters()

markID = 5

# Arduinoとのシリアル通信設定
arduino_port = 'COM4'  # Arduinoのポートに合わせて変更
arduino_baudrate = 115200
ser = serial.Serial(arduino_port, arduino_baudrate, timeout=1)

try:
    send_command_and_wait(ser, cap, 'serial_connecting', 'serial_connected')
    send_command_and_wait(ser, cap, 'motor_offset', 'processing_now')
    send_command_and_wait(ser, cap, 'none', 'processing_completed')
    time.sleep(2)

    send_command_and_wait(ser, cap, 'motorA_start', 'processing_now')

    # Rest of the code
    while True:
        date = get_mark_coordinate(cap, dict_aruco, parameters, markID)
        if date[1] < 180 +10:
            send_command_and_wait(ser, cap, 'motorA_stop', 'stop_motorA')
            send_command_and_wait(ser, cap, 'none', 'processing_completed')
            break

    
    send_command_and_wait(ser, cap, 'motorB_start', 'processing_now')

    # Rest of the code
    while True:
        date = get_mark_coordinate(cap, dict_aruco, parameters, markID)
        if date[0] < 350 +10:
            send_command_and_wait(ser, cap, 'motorB_stop', 'stop_motorB')
            send_command_and_wait(ser, cap, 'none', 'processing_completed')
            break

except Exception as e:
    print("Error:", str(e))


time.sleep(1)

print(' ----- get_mark_coordinate ----- ')
print(get_mark_coordinate(cap, dict_aruco, parameters, markID))

send_command_and_wait(ser, cap, 'process_end', 'end_return')
print('------process end------')

ser.close()
cap.release()

cv2.destroyAllWindows()


Arduino Response: serial_connected
Arduino Response: processing_now
Arduino Response: processing_completed
Arduino Response: processing_now
Arduino Response: stop_motorA
Arduino Response: processing_completed
Arduino Response: processing_now
Arduino Response: stop_motorB
Arduino Response: processing_completed
 ----- get_mark_coordinate ----- 
[345.5, 177.5]
Arduino Response: end_return
------process end------


In [1]:
import cv2
from cv2 import aruco
import numpy as np
import serial
import time

def get_mark_coordinate(cap, dict_aruco, parameters, num_id):
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    corners, ids, rejectedImgPoints = aruco.detectMarkers(gray, dict_aruco, parameters=parameters)

    if ids is not None and num_id in ids:
        index = np.where(ids == num_id)[0][0]
        cornerUL = corners[index][0][0]
        cornerUR = corners[index][0][1]
        cornerBR = corners[index][0][2]
        cornerBL = corners[index][0][3]

        center = [(cornerUL[0] + cornerBR[0]) / 2, (cornerUL[1] + cornerBR[1]) / 2]

        # Draw coordinates on the frame
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(frame, f'ID: {num_id}', (int(center[0]), int(center[1])), font, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
        cv2.putText(frame, f'Coord: ({center[0]:.2f}, {center[1]:.2f})', (10, 30), font, 0.5, (0, 0, 255), 1, cv2.LINE_AA)

        cv2.imshow('Camera Window', frame)

        return center

    # If ID is not found, display the original frame
    cv2.imshow('Camera Window', frame)

    return None

def send_command_and_wait(ser, cap, command, expected_response):
    cap.grab()
    while True:
        if command != "none":
            ser.write((command + '\n').encode())

        response = ser.readline().decode().strip()
        if response == expected_response:
            print("Arduino Response:", response)
            break

cameraID = 0
cap = cv2.VideoCapture(cameraID)
dict_aruco = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)
parameters = aruco.DetectorParameters()

markID = 5

# Arduinoとのシリアル通信設定
arduino_port = 'COM4'  # Arduinoのポートに合わせて変更
arduino_baudrate = 115200
ser = serial.Serial(arduino_port, arduino_baudrate, timeout=1)

try:
    send_command_and_wait(ser, cap, 'serial_connecting', 'serial_connected')
    send_command_and_wait(ser, cap, 'motor_offset', 'processing_now')
    send_command_and_wait(ser, cap, 'none', 'processing_completed')
    time.sleep(2)

    send_command_and_wait(ser, cap, 'motorAB_start', 'processing_now')
    flagA = flagB = False

    while True:
        date = get_mark_coordinate(cap, dict_aruco, parameters, markID)
        if date[1] < 180 +10:
            send_command_and_wait(ser, cap, 'motorA_stop', 'stop_motorA')
            flagA = True

        if date[0] < 350 +10:
            send_command_and_wait(ser, cap, 'motorB_stop', 'stop_motorB')
            flagB = True

        if flagA and flagB:
            break

        


except Exception as e:
    print("Error:", str(e))


time.sleep(1)

print(' ----- get_mark_coordinate ----- ')
print(get_mark_coordinate(cap, dict_aruco, parameters, markID))

send_command_and_wait(ser, cap, 'process_end', 'end_return')
print('------process end------')

ser.close()
cap.release()

cv2.destroyAllWindows()


Arduino Response: serial_connected
Arduino Response: processing_now
Arduino Response: processing_completed
Arduino Response: processing_now
Arduino Response: stop_motorA
